# Probabilidade

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

A base das estatísticas Bayesianas é o Teorema de Bayes, e a base do Teorema de Bayes é a probabilidade condicional.  

Neste capítulo, vamos começar com a probabilidade condicional, derivar o Teorema de Bayes, e demonstrá-lo utilizando um conjunto de dados real.  No próximo capítulo, utilizaremos o Teorema de Bayes para resolver problemas relacionados com a probabilidade condicional.  Nos capítulos que se seguem, faremos a transição do Teorema de Bayes para as estatísticas Bayesianas, e explicarei a diferença.

## Linda, a Banqueiro

Para introduzir a probabilidade condicional, vou utilizar um exemplo de um [famous experiment by Tversky and Kahneman](https://en.wikipedia.org/wiki/Conjunction_fallacy), que colocou a seguinte questão:

> Linda tem 31 anos de idade, é solteira, franca e muito brilhante. É licenciada em filosofia. Como estudante, estava profundamente preocupada com questões de discriminação e justiça social, e também participou em demonstrações anti-nucleares.  O que é mais provável?

> 1. Linda é caixa de banco.

> 2. Linda é uma caixa de banco e é activa no movimento feminista.

Muitas pessoas escolhem a segunda resposta, presumivelmente porque parece mais consistente com a descrição.  Parece pouco característico se Linda for *just* uma caixa de banco; parece mais consistente se ela for também uma feminista.

Mas a segunda resposta não pode ser "mais provável", como a pergunta faz.  Suponhamos que encontramos 1000 pessoas que correspondem à descrição de Linda e 10 delas trabalham como caixas bancárias.  Quantas delas são também feministas?  No máximo, todas as 10 são; nesse caso, as duas opções são *igualmente* prováveis.  Se menos de 10 forem, a segunda opção é *sem* probabilidade*.  Mas não há maneira de a segunda opção ser *mais* provável.

Se estava inclinado a escolher a segunda opção, está em boa companhia.  O biólogo [Stephen J. Gould wrote](https://doi.org/10.1080/09332480.1989.10554932) :


Se a pequena pessoa na sua cabeça ainda estiver infeliz, talvez este capítulo ajude.

## Probabilidade

Neste momento, devo fornecer uma definição de "probabilidade", mas que [turns out to be surprisingly difficult](https://en.wikipedia.org/wiki/Probability_interpretations).  Para evitar ficarmos presos antes de começarmos, utilizaremos uma definição simples por agora e aperfeiçoá-la-emos mais tarde: A **probabilidade** é uma fracção de um conjunto finito.

Por exemplo, se examinarmos 1000 pessoas, e 20 delas são caixas bancárias, a fracção que trabalha como caixas bancárias é de 0,02 ou 2\%.  Se escolhermos aleatoriamente uma pessoa desta população, a probabilidade de ela ser caixa de um banco é de 2\%.

Por "ao acaso" quero dizer que cada pessoa no conjunto de dados tem a mesma hipótese de ser escolhida.

Com esta definição e um conjunto de dados apropriado, podemos calcular as probabilidades através da contagem.

Para demonstrar, vou utilizar dados do [General Social Survey](http://gss.norc.org/) (GSS).  

A célula seguinte descarrega os dados.

In [1]:
# Load the data file

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/BiteSizeBayes/raw/master/gss_bayes.csv')

Vou utilizar Pandas para ler os dados e armazená-los num "DataFrame".

In [2]:
import pandas as pd

gss = pd.read_csv('gss_bayes.csv', index_col=0)
gss.head()

O "DataFrame" tem uma linha para cada pessoa inquirida e uma coluna para cada variável que seleccionei.

As colunas são

* 'caseid': ID do inquirido (que é o índice da tabela).

*`ano`: Ano em que o inquirido foi inquirido.

* "idade": Idade do inquirido ao ser inquirido.

* "sex": Macho ou fêmea.

*`polviews'': Opiniões políticas sobre um leque que vai do liberal ao conservador.

* 'partyid': Filiação em partidos políticos, Democrata, Independente, ou Republicano.

* `indus10`: [Code](https://www.census.gov/cgi-bin/sssd/naics/naicsrch?chart=2007) para a indústria em que o inquirido trabalha.

Vamos olhar para estas variáveis com mais detalhe, começando por `indus10`.

## Fracção de banqueiros

O código para "Banca e actividades relacionadas" é 6870, pelo que podemos seleccionar banqueiros como este:

In [3]:
banker = (gss['indus10'] == 6870)
banker.head()

O resultado é uma "Série" Pandas que contém os valores booleanos "Verdade" e "Falso".

Se utilizarmos a função "soma" nesta "Série", ela trata "Verdade" como 1 e "Falso" como 0, portanto o total é o número de banqueiros.

In [4]:
banker.sum()

Neste conjunto de dados, existem 728 banqueiros.

Para calcular a *fracção* dos banqueiros, podemos utilizar a função `mean`, que calcula a fracção de valores 'Verdade' na `Série':

In [5]:
banker.mean()

Cerca de 1,5% dos inquiridos trabalham na banca, portanto, se escolhermos uma pessoa aleatória do conjunto de dados, a probabilidade de eles serem banqueiros é de cerca de 1,5%.

## A Função de Probabilidade

Vou colocar o código da secção anterior numa função que pega numa série booleana e devolve uma probabilidade:

In [6]:
def prob(A):
    """Computes the probability of a proposition, A."""    
    return A.mean()

Assim, podemos calcular a fracção de banqueiros como este:

In [7]:
prob(banker)

Agora vamos olhar para outra variável neste conjunto de dados.

Os valores da coluna `sex` são codificados desta forma:

```
1    Male
2    Female
```

Assim, podemos fazer uma série booleana que seja "Verdadeira" para as inquiridas do sexo feminino e "Falsa" de outra forma.

In [8]:
female = (gss['sex'] == 2)

E utilizá-lo para calcular a fracção de inquiridos que são mulheres.

In [9]:
prob(female)




```
1	Extremely liberal
2	Liberal
3	Slightly liberal
4	Moderate
5	Slightly conservative
6	Conservative
7	Extremely conservative
```

Vou definir "liberal" para ser "verdadeiro" para qualquer pessoa cuja resposta seja "Extremamente liberal", "Liberal", ou "Ligeiramente liberal".

In [10]:
liberal = (gss['polviews'] <= 3)

Eis a fracção dos inquiridos que são liberais por esta definição.

In [11]:
prob(liberal)

Se escolhermos uma pessoa aleatória neste conjunto de dados, a probabilidade de ela ser liberal é de cerca de 27%.

Os valores do `partidário` são codificados desta forma:

```
0	Strong democrat
1	Not strong democrat
2	Independent, near democrat
3	Independent
4	Independent, near republican
5	Not strong republican
6	Strong republican
7	Other party
```

Vou definir "democrata" para incluir os inquiridos que escolheram "democrata forte" ou "não democrata forte":

In [12]:
democrat = (gss['partyid'] <= 1)

E aqui está a fracção dos inquiridos que são Democratas, por esta definição.

In [13]:
prob(democrat)

## Conjunção

Agora que temos uma definição de probabilidade e uma função que a computa, passemos à conjunção.

"Conjunção" é outro nome para a operação lógica "e".  Se tiver dois [propositions](https://en.wikipedia.org/wiki/Proposition), `A` e `B`, a conjunção `A e B` é `Verdade` se ambos `A` e `B` forem `Verdade`, e `Falso` caso contrário.

Se tivermos duas séries booleanas, podemos utilizar o operador `&` para calcular a sua conjunção.

Por exemplo, já calculámos a probabilidade de um inquirido ser um banqueiro.

In [14]:
prob(banker)

E a probabilidade de ser um democrata:

In [15]:
prob(democrat)

Agora podemos calcular a probabilidade de um inquirido ser um banqueiro *e* um democrata:

In [16]:
prob(banker & democrat)

Como seria de esperar, `prob(banqueiro & democrata)` é menos do que `prob(banqueiro)`, porque nem todos os banqueiros são democratas.



In [17]:
prob(democrat & banker)

Como esperado, eles são os mesmos.

## Probabilidade condicional




* Qual é a probabilidade de um inquirido ser liberal, dado que é do sexo feminino?

Comecemos com o primeiro, que podemos interpretar assim: "De todos os entrevistados que são liberais, que fracção são democratas?"

Podemos calcular esta probabilidade em duas etapas:


2. Calcule a fracção dos inquiridos seleccionados que são democratas.

Para seleccionar inquiridos liberais, podemos utilizar o operador de parênteses, `[]`, desta forma:

In [18]:
selected = democrat[liberal]

"seleccionado" contém os valores de "democrata" para os inquiridos liberais, por isso, "prob(seleccionado)" é a fracção de liberais que são democratas:

In [19]:
prob(selected)

Um pouco mais de metade dos liberais são democratas.  Se esse resultado for inferior ao esperado, tenha em mente:

1. Utilizámos uma definição algo rigorosa de "democrata", excluindo os Independentes que "se inclinam" para a democracia.

2. O conjunto de dados inclui inquiridos já em 1974; na primeira parte deste intervalo, havia menos alinhamento entre as opiniões políticas e a filiação partidária, em comparação com o presente.

Vamos tentar o segundo exemplo, "Qual é a probabilidade de um inquirido ser do sexo feminino, dado que é um banqueiro"?

Podemos interpretar isso para significar, "De todos os inquiridos que são banqueiros, que fracção são do sexo feminino"?

Mais uma vez, utilizaremos o operador de parênteses para seleccionar apenas os banqueiros e o `prob' para calcular a fracção que são do sexo feminino.

In [20]:
selected = female[banker]
prob(selected)

Cerca de 77% dos banqueiros deste conjunto de dados são do sexo feminino.




In [21]:
def conditional(proposition, given):
    """Probability of A conditioned on given."""
    return prob(proposition[given])

Podemos utilizar "condicional" para calcular a probabilidade de um inquirido ser liberal, dado que é do sexo feminino.

In [22]:
conditional(liberal, given=female)

Cerca de 28% das mulheres inquiridas são liberais.

Incluí a palavra-chave, "dado", juntamente com o parâmetro, "mulher", para tornar esta expressão mais legível.

## Probabilidade condicional não é comutativa

Vimos que a conjunção é comutativa; ou seja, `prob(A & B)` é sempre igual a `prob(B & A)`.

Mas a probabilidade condicional é *não* comutativa; ou seja, `conditional(A, B)` não é o mesmo que `conditional(B, A)`.

Isto deve ficar claro se olharmos para um exemplo.  Anteriormente, calculámos a probabilidade de um inquirido ser do sexo feminino, dado que é um banqueiro.

In [23]:
conditional(female, given=banker)

O resultado mostra que a maioria dos banqueiros são do sexo feminino.  Isto não é o mesmo que a probabilidade de um inquirido ser um banqueiro, dado que são do sexo feminino:

In [24]:
conditional(banker, given=female)

Apenas cerca de 2% das inquiridas do sexo feminino são banqueiras.

Espero que este exemplo torne claro que a probabilidade condicional não é comutativa, e talvez já estivesse claro para si.  No entanto, é um erro comum confundir `condicional(A, B)` e `condicional(B, A)`.  Veremos alguns exemplos mais tarde.

## Condição e Conjunção

Podemos combinar probabilidade condicional e conjunção.  Por exemplo, aqui está a probabilidade de um inquirido ser do sexo feminino, dado que é um democrata liberal.

In [25]:
conditional(female, given=liberal & democrat)

Cerca de 57% dos democratas liberais são do sexo feminino.

E aqui está a probabilidade de ser uma mulher liberal, dado que é uma banqueira:

In [26]:
conditional(liberal & female, given=banker)

Cerca de 17% dos banqueiros são mulheres liberais.

## Leis da Probabilidade

Nas próximas secções, vamos derivar três relações entre conjunção e probabilidade condicional:

* Teorema 1: Utilização de uma conjunção para calcular uma probabilidade condicional.

* Teorema 2: Utilização de uma probabilidade condicional para calcular uma conjunção.

* Teorema 3: Utilizando `conditional(A, B)` para calcular `conditional(B, A)`.

O teorema 3 é também conhecido como o Teorema de Bayes.

Escreverei estes teoremas usando notações matemáticas para a probabilidade:

* $P(A)$ é a probabilidade de proposta $A$.

* $P(A~\mathrm{and}~B)$ é a probabilidade da conjunção de $A$ e $B$, ou seja, a probabilidade de ambos serem verdadeiros.


Com isso, estamos prontos para o Theorem 1.

### Teorema 1

Que fracção de banqueiros são do sexo feminino?  Já vimos uma forma de calcular a resposta:

1. Utilizar o operador de suporte para seleccionar os banqueiros, depois

2. Utilizar `mean` para calcular a fracção de banqueiros que são do sexo feminino.

Podemos escrever estes passos desta forma:

In [27]:
female[banker].mean()

Ou podemos utilizar a função `condicional', que faz a mesma coisa:

In [28]:
conditional(female, given=banker)


1. A fracção de inquiridos que são mulheres banqueiras, e

2. A fracção dos inquiridos que são banqueiros.

Por outras palavras: de todos os banqueiros, que fracção são banqueiros do sexo feminino?



In [29]:
prob(female & banker) / prob(banker)

O resultado é o mesmo.  Este exemplo demonstra uma regra geral que relaciona a probabilidade condicional e a conjunção.  Eis o que parece na notação matemática:


E esse é o Teorema 1.




Esta fórmula sugere uma segunda forma de calcular uma conjunção: em vez de utilizar o operador `&`, podemos calcular o produto de duas probabilidades.



In [30]:
prob(liberal & democrat)

E aqui está o resultado usando o Teorema 2:

In [31]:
prob(democrat) * conditional(liberal, democrat)

São a mesma coisa.

### Teorema 3

Estabelecemos que a conjunção é comutativa.  Em notação matemática, isso significa:

$$P(A~\mathrm{and}~B) = P(B~\mathrm{and}~A)$$$

Se aplicarmos o Teorema 2 a ambos os lados, temos


Eis uma maneira de interpretar isso: se quiser verificar $A$ e $B$, pode fazê-lo em qualquer ordem:

1. Pode verificar primeiro $B$, depois $A$ condicionados a $B$, ou

2. Pode verificar primeiro $A$, depois $B$ condicionados a $A$.

Se dividirmos por $P(B)$, obtemos Theorem 3:


E isso, meus amigos, é o Teorema de Bayes.

Para ver como funciona, vamos calcular a fracção de banqueiros que são liberais, utilizando em primeiro lugar `condicional':

In [32]:
conditional(liberal, given=banker)

Agora usando o Teorema de Bayes:

In [33]:
prob(liberal) * conditional(banker, liberal) / prob(banker)

São a mesma coisa.


Para além destes três teoremas, há mais uma coisa que teremos de fazer com as estatísticas Bayesianas: a lei da probabilidade total.

Aqui está uma forma da lei, expressa em notação matemática:


Em palavras, a probabilidade total de $A$ é a soma de duas possibilidades: ou $B_1$ e $A$ são verdadeiros ou $B_2$ e $A$ são verdadeiros.

Mas esta lei só se aplica se $B_1$ e $B_2$ forem:


* Colectivamente exaustiva, o que significa que uma delas deve ser verdadeira.

Como exemplo, vamos utilizar esta lei para calcular a probabilidade de um inquirido ser um banqueiro.



In [34]:
prob(banker)

Portanto, vamos confirmar que obtemos a mesma coisa se calcularmos os banqueiros masculinos e femininos separadamente.

Neste conjunto de dados, todos os inquiridos são designados por homens ou mulheres.  Recentemente, o Conselho de Supervisores da GSS anunciou que acrescentará questões de género mais inclusivas ao inquérito (pode ler mais sobre esta questão, e a sua decisão, em <https://gender.stanford.edu/news-publications/gender-news/more-inclusive-gender-questions-added-general-social-survey>.


Aqui está a "Série" complementar para os respondentes masculinos.

In [35]:
male = (gss['sex'] == 1)

Agora podemos calcular a probabilidade total de um "banqueiro" como este. 

In [36]:
prob(male & banker) + prob(female & banker)

Porque "homem" e "mulher" são mutuamente exclusivos e colectivamente exaustivos (MECE), obtemos o mesmo resultado através do cálculo da probabilidade de "banqueiro" directamente.

Aplicando o Teorema 2, também podemos escrever a lei da probabilidade total desta forma:


E podemos testá-lo com o mesmo exemplo:

In [37]:
(prob(male) * conditional(banker, given=male) +
prob(female) * conditional(banker, given=female))

Quando existem mais de duas condições, é mais conciso escrever a lei da probabilidade total como uma soma:





In [38]:
B = gss['polviews']
B.value_counts().sort_index()

Nesta escala, `4.0` representa "Moderado".



In [39]:
i = 4
prob(B==i) * conditional(banker, B==i)

E podemos utilizar `sum` e um [generator expression](https://www.johndcook.com/blog/2020/01/15/generator-expression/) para calcular a soma.

In [40]:
sum(prob(B==i) * conditional(banker, B==i)
    for i in range(1, 8))

O resultado é o mesmo.

Neste exemplo, utilizar a lei da probabilidade total é muito mais trabalho do que calcular directamente a probabilidade, mas vai revelar-se útil, prometo.

## Resumo

Eis o que temos até agora:

**Theorem 1** dá-nos uma forma de calcular uma probabilidade condicional usando uma conjunção:

$$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$$$







Nesta altura pode perguntar: "E depois?".  Se tivermos todos os dados, podemos calcular qualquer probabilidade que quisermos, qualquer conjunção, ou qualquer probabilidade condicional, apenas contando.  Não temos de utilizar estas fórmulas.

E tem razão, *se* tivermos todos os dados.  Mas muitas vezes não temos, e nesse caso, estas fórmulas podem ser bastante úteis - especialmente o Teorema de Bayes.



## Exercícios

**Exercício:** Vamos utilizar as ferramentas deste capítulo para resolver uma variação do problema de Linda.

> Linda tem 31 anos de idade, é solteira, franca e muito brilhante. É licenciada em filosofia. Como estudante, estava profundamente preocupada com questões de discriminação e justiça social, e também participou em demonstrações anti-nucleares.  O que é mais provável?

> 1. Linda é uma banqueira.


Para responder a esta pergunta, calcular 

* A probabilidade de Linda ser uma banqueira feminina,

* A probabilidade de Linda ser uma banqueira liberal, e

* A probabilidade de Linda ser uma banqueira liberal feminina e uma democrata.

In [41]:
# Solution goes here

In [42]:
# Solution goes here

In [43]:
# Solution goes here

**Exercício:** Utilizar `condicional` para calcular as seguintes probabilidades:

* Qual é a probabilidade de um inquirido ser liberal, dado que é um democrata?

* Qual é a probabilidade de um inquirido ser um democrata, dado que é liberal?

Pense cuidadosamente sobre a ordem dos argumentos que passa a "condicional".

In [44]:
# Solution goes here

In [45]:
# Solution goes here


> Se não for um liberal aos 25 anos, não tem coração. Se não for um conservador aos 35 anos, não tem cérebro.

Quer concorde ou não com esta proposta, ela sugere algumas probabilidades que podemos calcular como um exercício.



In [46]:
young = (gss['age'] < 30)
prob(young)

In [47]:
old = (gss['age'] >= 65)
prob(old)

Para estes limiares, escolhi números redondos próximos do 20º e 80º percentis.  Dependendo da sua idade, pode ou não concordar com estas definições de "jovem" e "velho".



In [48]:
conservative = (gss['polviews'] >= 5)
prob(conservative)

Utilizar `prob` e `condicional` para calcular as seguintes probabilidades.

* Qual é a probabilidade de um inquirido escolhido aleatoriamente ser um jovem liberal?

* Qual é a probabilidade de um jovem ser liberal?

* Que fracção dos inquiridos são velhos conservadores?

* Que fracção de conservadores são antigos?

Para cada declaração, pensar se ela exprime uma conjunção, uma probabilidade condicional, ou ambas.

Para as probabilidades condicionais, tenha cuidado com a ordem dos argumentos.

Se a sua resposta à última pergunta for superior a 30%, tem-na ao contrário!

In [49]:
# Solution goes here

In [50]:
# Solution goes here

In [51]:
# Solution goes here

In [52]:
# Solution goes here